In [4]:
const padHeaderLines = (match) => {
	const padLine = (line, length) => {
		const padLength = length - line.length;
		if (padLength > 0) {
			const [padLeft, padRight] = [Math.ceil(padLength / 2), Math.ceil(padLength / 2)];
			const [lineLeft, lineRight] = [
				line.slice(0, Math.floor(line.length / 2)),
				line.slice(Math.floor(line.length / 2))
			];
			// Two types of padding: '█' and '░'. Count amount of each to get relative ratio.
			const fadePad = lineLeft.match(/░+/u)?.pop().length ?? 0;
			const fullFadeRatio = fadePad === 0 ? 1 : (lineLeft.match(/░+/)?.pop().length ?? 0) / fadePad;
			let numFullPadLeft = Math.round((fullFadeRatio * padLeft) / (1 + fullFadeRatio)),
							numFadePadLeft = 0,
							numFullPadRight = Math.round((fullFadeRatio * padRight) / (1 + fullFadeRatio)),
							numFadePadRight = 0;
			if (fadePad > 0) {
				numFadePadLeft = padLeft - numFullPadLeft;
				numFadePadRight = padRight - numFullPadRight;
			} else {
				numFullPadLeft = padLeft;
				numFullPadRight = padRight;
			}
			numFullPadRight += padLength - (numFullPadLeft + numFadePadLeft + numFullPadRight + numFadePadRight);
			return [
				lineLeft.replace(/▌█/u, `▌${"█".repeat(numFullPadLeft + 1)}`).replace(/░/u, "░".repeat(numFadePadLeft + 1)),
				lineRight.replace(/█▐/u, `${"█".repeat(numFullPadRight + 1)}▐`).replace(/░/u, "░".repeat(numFadePadRight + 1))
			].join("");
		}
		return line;
	};
	const lines = match.split(/\n/s);
	const returnLines = [];
	let [maxIndex, maxLen] = [0, 0];
	lines.forEach((line, i) => {
		if (line.length > maxLen) {
			maxIndex = i;
			maxLen = line.length;
		}
	});
	lines.forEach((line) => {
		if (line.length < maxLen) {
			returnLines.push(padLine(line, maxLen));
		} else {
			returnLines.push(line);
		}
	});
	return returnLines.join("\n");
};
const multiLineStripper = (match, startBreak, contents, endBreak) => {
	

};
const REGEXPPATTERNS = new Map([
	[/(\r\n)?\s*?(\/\*DEVCODE\*\/.*?\/\*!DEVCODE\*\/)\s*?(\r\n)/gsm, multiLineStripper], // Strip developer code
	[/\/\* \*{4}▌.*?▐\*{4} \*\//s, padHeaderLines], // Pad header lines to same length
	[/(\r?\n?)[ \t]*\/\*{1,2}(?!~)(?:.|\r?\n|\r)*?\*\/[ \t]*(\r?\n?)/g, "$1$2"], // Strip multi-line comments unless they beginning with '/*~' or '/**~'
	[/(\r?\n?)[ \t]*\/\/~.*?$/gm, "$1"], // Strip single-line comments unless they begin with '//~'
	[/[ \t]*\/\/[ \t]*eslint.*$/gm, ""], // Strip eslint enable/disable single-line comments
	[/[ \t]*\/\/[ \t]*@ts.*$/gm, ""], // Strip TypeScript expect-error comments
	[/[ \t]*\/\*[ \t]*eslint[^*]*\*\/[ \t]*/g, ""], // Strip eslint enable/disable mult-line comments
	[/[ \t]*\/\/ no default.*$/gm, ""], // Strip '// no default'
	[/[ \t]*\/\/ falls through.*$/gm, ""], // Strip '// falls through'
	[/[ \t]*~$/gm, ""], // Strip '~' from end-of-lines (used for automatic region folding)
	[/#reg.*? /gs, ""], // Convert region headers to standard headers
	[/(\r?\n?)[ \t]*\/\/ #endreg.*[ \t]*\r?\n?/g, "\r\n"], // Remove region footers
	[/(\r?\n[ \t]*(?=\r?\n)){2,}/g, "\r\n"], // Strip excess blank lines
	[/[ \t]*\r?\n$/g, ""], // Strip whitespace from end of files
	[/^[ \t]*\r?\n/g, ""] // Strip whitespace from start of files
]);
console.log(REGEXPPATTERNS);

Map(15) {
  /(\r\n)?\s*?(\/\*DEVCODE\*\/.*?\/\*!DEVCODE\*\/)\s*?(\r\n)/gms => [Function: multiLineStripper],
  /\/\* \*{4}▌.*?▐\*{4} \*\//s => [Function: padHeaderLines],
  /(\r?\n?)[ \t]*\/\*{1,2}(?!~)(?:.|\r?\n|\r)*?\*\/[ \t]*(\r?\n?)/g => '$1$2',
  /(\r?\n?)[ \t]*\/\/~.*?$/gm => '$1',
  /[ \t]*\/\/[ \t]*eslint.*$/gm => '',
  /[ \t]*\/\/[ \t]*@ts.*$/gm => '',
  /[ \t]*\/\*[ \t]*eslint[^*]*\*\/[ \t]*/g => '',
  /[ \t]*\/\/ no default.*$/gm => '',
  /[ \t]*\/\/ falls through.*$/gm => '',
  /[ \t]*~$/gm => '',
  /#reg.*? /gs => '',
  /(\r?\n?)[ \t]*\/\/ #endreg.*[ \t]*\r?\n?/g => '\r\n',
  /(\r?\n[ \t]*(?=\r?\n)){2,}/g => '\r\n',
  /[ \t]*\r?\n$/g => '',
  /^[ \t]*\r?\n/g => ''
}


In [5]:
// The HTML representation of an instance of GameObject, defined in game API
class DOMObject {
	private readonly gameObj: GameObject; // The linked GameObject, with the render method
	protected onRender: { // stores data to be applied when the DOMObject is rendered
		set?: gsap.CSSProperties // style properties that can't be applied until object is rendered
		animateTo?: gsap.TweenVars // properties of animation to be fired on render
	}

	// ... etc ...

	public async renderObject(): Promise<void> {
		this.renderPromise = this.gameObj.renderToDOM(); // GameObject's async render method
		await this.renderPromise; // wait for the render to complete, then apply onRender data
		if (this.onRender.animateTo) {
			gsap.to(this.element, this.onRender.animateTo);
		}
		return this.renderPromise;
	}

	public animateTo(vars: gsap.TweenVars): gsap.core.Tween | false {
		if (this.gameObj.isRendered) {
			return gsap.to(this.element, vars) // returns a Tween, but only if it's rendered
		}
		this.onRender.animateTo = { // otherwise, updates onRender so the animation can
			...this.onRender.animateTo ?? {}, //      be fired when it is rendered ...
			...vars
		}
		return false; // ... but I can't return a Tween in this case.
	}
}

SyntaxError: Unexpected identifier

In [1]:
// U.OBJMERGE TESTING
const U = (() => {
	const isNumber = (ref) => typeof ref === "number" && !isNaN(ref);
	const isList = (ref) => ref && Object.getPrototypeOf(ref) === Object.prototype;
	const isArray = (ref) => Array.isArray(ref);
	const isFunc = (ref) => typeof ref === "function";
	const isInt = (ref) => isNumber(ref) && Math.round(ref) === ref;
	const isFloat = (ref) => isNumber(ref) && Math.round(ref) !== ref;
	const isPosInt = (ref) => isInt(ref) && ref >= 0;
	const isIndex = (ref) => isList(ref) || isArray(ref);
	const isIterable = (ref) => typeof ref === "object" && ref !== null && Symbol.iterator in ref;
	const isHTMLCode = (ref) => typeof ref === "string" && /^<.*>$/u.test(ref);
	const isUndefined = (ref) => ref === undefined;
	const isDefined = (ref) => !isUndefined(ref);
	const isEmpty = (ref) => Object.keys(ref).length === 0;
	const hasItems = (ref) => Object.keys(ref).length > 0;
	const areEqual = (...refs) => {
		function checkEquality(ref1, ref2) {
			if (typeof ref1 !== typeof ref2) {
				return false;
			}
			if ([ref1, ref2].includes(null)) {
				return ref1 === ref2;
			}
			switch (typeof ref1) {
				case "object": {
					if (isArray(ref1)) {
						if (!isArray(ref2)) {
							return false;
						}
						if (ref1.length !== ref2.length) {
							return false;
						}
						for (let i = 0; i < ref1.length; i++) {
							if (!checkEquality(ref1[i], ref2[i])) {
								return false;
							}
						}
						return true;
					} else if (isList(ref1)) {
						if (!isList(ref2) || Object.keys(ref1).length !== Object.keys(ref2).length) {
							return false;
						}
						return checkEquality(Object.keys(ref1), Object.keys(ref2)) && checkEquality(Object.values(ref1), Object.values(ref2));
					}
					try {
						return JSON.stringify(ref1) === JSON.stringify(ref2);
					} catch {
						return false;
					}
				}
				default: {
					return ref1 === ref2;
				}
			}
		}
		let ref = refs.pop();
		while (refs.length) {
			if (checkEquality(ref, refs[0])) {
				ref = refs.pop();
			} else {
				return false;
			}
		}
		return true;
	};
	const pFloat = (ref, sigDigits, isStrict = false) => {
		if (typeof ref === "string") {
			ref = parseFloat(ref);
		}
		if (typeof ref === "number") {
			if (isNaN(ref)) {
				return isStrict ? NaN : 0;
			}
			if (isUndefined(sigDigits)) {
				return ref;
			}
			return Math.round(ref * (10 ** sigDigits)) / (10 ** sigDigits);
		}
		return isStrict ? NaN : 0;
	};
	const pInt = (ref, isStrict = false) => (isNaN(pFloat(ref, 0, isStrict)) ? NaN : Math.round(pFloat(ref, 0, isStrict)));
	const objFilter = (obj, keyFunc, valFunc) => {
		// An object-equivalent Array.filter() function, which accepts filter functions for both keys and/or values.
		// If only one function is provided, it's assumed to be mapping the values and will receive (v, k) args.
		if (!valFunc) {
			valFunc = keyFunc;
			keyFunc = false;
		}
		if (!keyFunc) {
			keyFunc = ((k) => k);
		}
		if (isArray(obj)) {
			return obj.filter(valFunc);
		}
		const kFunc = keyFunc || (() => true);
		const vFunc = valFunc || (() => true);
		// @ts-expect-error TEMPORARY
		return Object.fromEntries(Object.entries(obj).filter(([key, val]) => kFunc(key) && vFunc(val)));
	};
	const objCompact = (obj, preserve = []) => objFilter(obj, (val) => preserve.includes(`${val}`));
	const objClone = (obj, isStrictlySafe = false) => {
		let cloneObj;
		try {
			cloneObj = JSON.parse(JSON.stringify(obj));
		} catch (err) {
			if (isStrictlySafe) {
				throw new Error(`${err}`);
			}
			if (isIterable(obj)) {
				cloneObj = {...obj};
			}
		}
		return cloneObj;
	};
	function objMerge(target, source, {isMutatingOk = false, isStrictlySafe = false, isConcatenatingArrays = true} = {}) {
		/* Returns a deep merge of source into target. Does not mutate target unless isMutatingOk = true. */
		target = isMutatingOk ? target : objClone(target, isStrictlySafe);
		if (isUndefined(target)) {
			return objClone(source);
		}
		if (isUndefined(source)) {
			return target;
		}
		if (isIndex(source)) {
			for (const [key, val] of Object.entries(source)) {
				if (isConcatenatingArrays && isArray(target[key]) && isArray(val)) {
					target[key] = [...target[key], ...val];
				} else if (val !== null && typeof val === "object") {
					// @ts-expect-error TEMPORARY
					if (target[key] === undefined) {
						// @ts-expect-error TEMPORARY
						target[key] = Array.isArray(val) ? [] : new (Object.getPrototypeOf(val).constructor());
					}
					// @ts-expect-error TEMPORARY
					target[key] = objMerge(target[key], val, {isMutatingOk: true, isStrictlySafe});
				} else {
					// @ts-expect-error TEMPORARY
					target[key] = val;
				}
			}
		}
		return target;
	}
	const objExpand = (obj) => {
		const expObj = {};
		for (let [key, val] of Object.entries(obj)) {
			if (isList(val)) {
				val = objExpand(val);
			}
			expObj[key] = val;
		}
		return expObj;
	};
	const objFlatten = (obj) => {
		const flatObj = {};
		for (const [key, val] of Object.entries(obj)) {
			if ((isArray(val) || isList(val)) && hasItems(val)) {
				for (const [subKey, subVal] of Object.entries(objFlatten(val))) {
					flatObj[`${key}.${subKey}`] = subVal;
				}
			} else {
				flatObj[key] = val;
			}
		}
		return flatObj;
	};

	return {
		hasItems, areEqual, objFilter, objCompact, objClone, objMerge, objExpand, objFlatten
	}
})();
const targetObj = {
	"width": null,
	"popOut": false,
	"resizable": false,
	"id": "",
	"classes": ["x-item", "x-group"],
};
const testObjs = [
	{
		"resizable": undefined,
		"classes": ["x-die"],
		"newEntry": null
	}
];

const TESTS = [
	{
		function: U.objMerge,
		params: [targetObj, testObjs[0], {isConcatenatingArrays: true}]
	},
	{
		function: U.objMerge,
		params: [targetObj, testObjs[0], {isConcatenatingArrays: false}]
	},
	{
		function: U.objFlatten,
		params: [targetObj]
	},
	{
		function: U.objFlatten,
		params: [testObjs[0]]
	}
];

TESTS.forEach((testData) => {
	console.log(JSON.stringify(testData.function(...testData.params), null, 2));
});

{
  "width": null,
  "popOut": false,
  "id": "",
  "classes": [
    "x-item",
    "x-group",
    "x-die"
  ],
  "newEntry": null
}
{
  "width": null,
  "popOut": false,
  "id": "",
  "classes": [
    "x-die",
    "x-group"
  ],
  "newEntry": null
}
{
  "width": null,
  "popOut": false,
  "resizable": false,
  "id": "",
  "classes.0": "x-item",
  "classes.1": "x-group"
}
{
  "classes.0": "x-die",
  "newEntry": null
}
